<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Creating gs://cloud-training-demos-ml/...
ServiceException: 409 Bucket cloud-training-demos-ml already exists.


<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,6392072535155213407,323758
1,8387817883864991792,331629
2,8391424625589759186,364497
3,9183605629983195042,329975
4,328012383083104805,359891


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13221 examples in the train dataset and 3328 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [7]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.125340,False,36,1,39.0,774501970389208065
1,9.449013,True,32,1,39.0,774501970389208065
2,7.059202,False,24,1,39.0,774501970389208065
3,8.126239,True,28,1,39.0,774501970389208065
4,5.436599,False,24,1,35.0,774501970389208065


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13206.000000,13221.000000,13221.000000,13128.000000,1.322100e+04
mean,7.213913,27.451479,1.035020,38.569698,4.387442e+18
std,1.330473,6.208165,0.192677,2.634689,2.807606e+18
min,0.500449,13.000000,1.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.060100,32.000000,1.000000,40.000000,7.108882e+18
max,12.090150,50.000000,4.000000,47.000000,9.183606e+18


In [9]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.125340,False,36,Single(1),39.0,774501970389208065
1,9.449013,True,32,Single(1),39.0,774501970389208065
2,7.059202,False,24,Single(1),39.0,774501970389208065
3,8.126239,True,28,Single(1),39.0,774501970389208065
4,5.436599,False,24,Single(1),35.0,774501970389208065


In [11]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13216,5.249206,Unknown,34,Single(1),37.0,6637442812569910270
13217,8.688418,Unknown,40,Single(1),40.0,6637442812569910270
13218,9.063204,Unknown,26,Single(1),43.0,6637442812569910270
13219,6.188376,Unknown,25,Single(1),37.0,6637442812569910270
13220,7.561856,Unknown,27,Single(1),39.0,6637442812569910270


In [12]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26230.000000,26230.000000,26230.000000,2.623000e+04
mean,7.216224,27.454213,38.577507,4.386282e+18
std,1.328838,6.209536,2.608520,2.806738e+18
min,0.500449,13.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.108882e+18
max,12.090150,50.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [13]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [14]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6602 eval.csv
  26230 train.csv
  32832 total
==> eval.csv <==
7.50012615324,False,20,Single(1),39.0,1088037545023002395
5.61517381314,True,23,Single(1),41.0,1891060869255459203
7.20691134478,True,20,Single(1),40.0,3182182455926341111
7.43839671988,False,22,Single(1),40.0,1639186255933990135
6.24349125984,False,31,Single(1),40.0,6910174677251748583
6.1244416383599996,False,32,Single(1),37.0,7146494315947640619
8.99926953484,True,24,Single(1),40.0,6910174677251748583
1.67771781382,True,19,Single(1),26.0,3182182455926341111
7.50012615324,True,35,Single(1),39.0,270792696282171059
6.93794738514,True,22,Single(1),38.0,74931465496927487

==> train.csv <==
7.12534030784,False,36,Single(1),39.0,774501970389208065
9.449012549319999,True,32,Single(1),39.0,774501970389208065
7.0592016292399995,False,24,Single(1),39.0,774501970389208065
8.12623897732,True,28,Single(1),39.0,774501970389208065
5.43659938092,False,24,Single(1),35.0,774501970389208065
7.36784879604,False,26,Single(1),42.0,774501970

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License